El nostre objectiu és estimar la CANTIDADCOMPRA del 2023. Per aquest motiu, només farem servir dades fins el 2022. Intentarem construir un dataset on les variables per entrenar ens puguin donar informació sobre la CANTIDADCOMPRA de l'any vinent.

In [368]:
import pandas as pd
import datetime as dt
from sklearn.metrics import mean_squared_error
import numpy as np

### Carreguem dataset

In [369]:
df = pd.read_excel('consumo_material_clean.xlsx')
df.head()

,CODIGO,FECHAPEDIDO,NUMERO,REFERENCIA,CANTIDADCOMPRA,UNIDADESCONSUMOCONTENIDAS,PRECIO,IMPORTELINEA,TIPOCOMPRA,ORIGEN,TGL,PRODUCTO
0,E99808,01/01/23,1595724/23,178567.1,60,10,62.59,375.54,Compra menor,1-2-60,TRANSITO,APOSITO DE FIBRAS DE POLIACRILATO C/PLATA-3
1,B41691,01/02/16,72714/16,400403,40,10,102.80,411.21,Compra menor,0-10-1,ALMACENABLE,SOLUCION P/ LIMPIEZA Y DESCONTAMINACION DE HER...
2,E64543,01/02/16,71961/16,403770,20,5,12.10,48.40,Compra menor,0-4-111,TRANSITO,APOSITO DE HIDROFIBRA / CINTA-18
3,E65007,01/02/16,72773/16,20415,100,50,215.32,430.65,Concurso,0-10-1,ALMACENABLE,APOSITO DE ESPUMA POLIURETANO / SACRO-11
4,E64911,01/02/17,86159/17,20701,300,300,792.00,792.00,Concurso,0-6-1,ALMACENABLE,APOSITO C/ CARBON Y PLATA-6


### Preprocessing

Canviem el format ed la columna FECHAPEDIDO per a q sigui un objecte datetime.

Creem dos columnes noves, una per l'any i una pel mes

In [370]:
df["FECHAPEDIDO"] = pd.to_datetime(df['FECHAPEDIDO'], format='%d/%m/%y')
df["year"] = df["FECHAPEDIDO"].dt.year
df["month"] = df["FECHAPEDIDO"].dt.month
df.head(2)

,CODIGO,FECHAPEDIDO,NUMERO,REFERENCIA,CANTIDADCOMPRA,UNIDADESCONSUMOCONTENIDAS,PRECIO,IMPORTELINEA,TIPOCOMPRA,ORIGEN,TGL,PRODUCTO,year,month
0,E99808,2023-01-01,1595724/23,178567.1,60,10,62.59,375.54,Compra menor,1-2-60,TRANSITO,APOSITO DE FIBRAS DE POLIACRILATO C/PLATA-3,2023,1
1,B41691,2016-02-01,72714/16,400403,40,10,102.80,411.21,Compra menor,0-10-1,ALMACENABLE,SOLUCION P/ LIMPIEZA Y DESCONTAMINACION DE HER...,2016,2


Eliminem les columnes q no utilitzarem.

Agrupem les files que tinguin el mateix valor en les columnes de l'any, mes, origen i codi. Fent la suma de la columna CANTIDADCOMPRA de totes aquestes files q agrupem en una.

In [371]:
df = df.drop(columns=["FECHAPEDIDO","PRECIO", "IMPORTELINEA", "UNIDADESCONSUMOCONTENIDAS", "NUMERO", "REFERENCIA", "TIPOCOMPRA", "TGL", "PRODUCTO"])
df = df.groupby(["year", "month", "ORIGEN", "CODIGO"]).sum().reset_index()
df.head(2)

,year,month,ORIGEN,CODIGO,CANTIDADCOMPRA
0,2015,1,0-0-2,B41691,100
1,2015,1,0-0-2,E64663,250


Ordenem el dataset segons les columnes: 'year', 'month', 'ORIGEN', 'CODIGO'.

Codi molt cutre per afegir les següents columnes al dataset q tenim fins ara:
- **previous_year_CANTIDADCOMPRA**: valor de CANTIDADCOMPRA l'any anterior per les mateixes característiques(mateix mes, mateix codi de producte i origen)
- **previous_year2_CANTIDADCOMPRA**: com l'anterior però enlloc de l'any anterior, de fa dos anys enrere
- **previous_month_CANTIDADCOMPRA**: lo mateix però enlloc de mirar l'any anterior mirem el mes anterior
- ... (fem el mateix per dos i tres mesos enrere)
- **next_year_CANTIDADCOMPRA**: el mateix però amb l'any següent

Això ho fem per tenir en compte d'alguna manera la temporalitat

Per últim afegim una variable covid on definim 3 nivells difernts en funció de l'estat de la pandemia, per distingir quan hi havia algun repunt important de casos, xq considerem q pot aportar info rellevant.

In [372]:
df.sort_values(by=['year', 'month', 'ORIGEN', 'CODIGO'], inplace=True)

# Add a new column 'previous_year_CANTIDADCOMPRA' with NaN values
df['previous_year_CANTIDADCOMPRA'] = 0
df['previous_year2_CANTIDADCOMPRA'] = 0
df['previous_month_CANTIDADCOMPRA'] = 0
df['previous_month2_CANTIDADCOMPRA'] = 0
df['previous_month3_CANTIDADCOMPRA'] = 0
df['next_year_CANTIDADCOMPRA'] = 0
df['date'] = pd.to_datetime(df[['year', 'month']].assign(DAY=1))
covid_1_i = dt.datetime(year=2020, month=3, day=1)
covid_1_f = dt.datetime(year=2020, month=6, day=1)
covid_2_i = dt.datetime(year=2020, month=8, day=1)
covid_2_f = dt.datetime(year=2021, month=5, day=1)

# Iterate through each row and populate the 'previous_year_CANTIDADCOMPRA' column
for index, row in df.iterrows():

    if covid_1_i < row['date'] < covid_1_f:
        covid = 1
    elif covid_2_i < row['date'] < covid_2_f:
        covid = 2
    else:
        covid = 0
    
    df.at[index, 'covid'] = covid

    previous_year_data = df.loc[
        (df['year'] == row['year'] - 1) &
        (df['month'] == row['month']) &
        (df['ORIGEN'] == row['ORIGEN']) &
        (df['CODIGO'] == row['CODIGO'])
    ]

    previous_year_data2 = df.loc[
        (df['year'] == row['year'] - 2) &
        (df['month'] == row['month']) &
        (df['ORIGEN'] == row['ORIGEN']) &
        (df['CODIGO'] == row['CODIGO'])
    ]

    next_year_data = df.loc[
        (df['year'] == row['year'] + 1) &
        (df['month'] == row['month']) &
        (df['ORIGEN'] == row['ORIGEN']) &
        (df['CODIGO'] == row['CODIGO'])
    ]

    if row["month"] == 1:
        previous_month_data = df.loc[
        (df['year'] == row['year'] - 1) &
        (df['month'] == 12) &
        (df['ORIGEN'] == row['ORIGEN']) &
        (df['CODIGO'] == row['CODIGO'])
        ]
        previous_month_data2 = df.loc[
        (df['year'] == row['year'] - 1) &
        (df['month'] == 11) &
        (df['ORIGEN'] == row['ORIGEN']) &
        (df['CODIGO'] == row['CODIGO'])
        ]
        previous_month_data3 = df.loc[
        (df['year'] == row['year'] - 1) &
        (df['month'] == 10) &
        (df['ORIGEN'] == row['ORIGEN']) &
        (df['CODIGO'] == row['CODIGO'])
        ]
    elif row["month"] == 2:
        previous_month_data = df.loc[
        (df['year'] == row['year'] - 1) &
        (df['month'] == 1) &
        (df['ORIGEN'] == row['ORIGEN']) &
        (df['CODIGO'] == row['CODIGO'])
        ]
        previous_month_data2 = df.loc[
        (df['year'] == row['year'] - 1) &
        (df['month'] == 12) &
        (df['ORIGEN'] == row['ORIGEN']) &
        (df['CODIGO'] == row['CODIGO'])
        ]
        previous_month_data3 = df.loc[
        (df['year'] == row['year'] - 1) &
        (df['month'] == 11) &
        (df['ORIGEN'] == row['ORIGEN']) &
        (df['CODIGO'] == row['CODIGO'])
        ]
    elif row["month"] == 3:
        previous_month_data = df.loc[
        (df['year'] == row['year'] - 1) &
        (df['month'] == 2) &
        (df['ORIGEN'] == row['ORIGEN']) &
        (df['CODIGO'] == row['CODIGO'])
        ]
        previous_month_data2 = df.loc[
        (df['year'] == row['year'] - 1) &
        (df['month'] == 1) &
        (df['ORIGEN'] == row['ORIGEN']) &
        (df['CODIGO'] == row['CODIGO'])
        ]
        previous_month_data3 = df.loc[
        (df['year'] == row['year'] - 1) &
        (df['month'] == 12) &
        (df['ORIGEN'] == row['ORIGEN']) &
        (df['CODIGO'] == row['CODIGO'])
        ]
    else:
        previous_month_data = df.loc[
        (df['year'] == row['year'] - 1) &
        (df['month'] == row['month'] - 1) &
        (df['ORIGEN'] == row['ORIGEN']) &
        (df['CODIGO'] == row['CODIGO'])
        ]
        previous_month_data2 = df.loc[
        (df['year'] == row['year'] - 1) &
        (df['month'] == row['month'] - 2) &
        (df['ORIGEN'] == row['ORIGEN']) &
        (df['CODIGO'] == row['CODIGO'])
        ]
        previous_month_data3 = df.loc[
        (df['year'] == row['year'] - 1) &
        (df['month'] == row['month'] - 3) &
        (df['ORIGEN'] == row['ORIGEN']) &
        (df['CODIGO'] == row['CODIGO'])
        ]

    # Check if there's a matching row in the previous year
    if not previous_year_data.empty:
        previous_year_CANTIDADCOMPRA = previous_year_data.iloc[0]['CANTIDADCOMPRA']
        df.at[index, 'previous_year_CANTIDADCOMPRA'] = previous_year_CANTIDADCOMPRA
    if not previous_year_data2.empty:
        previous_year2_CANTIDADCOMPRA = previous_year_data2.iloc[0]['CANTIDADCOMPRA']
        df.at[index, 'previous_year2_CANTIDADCOMPRA'] = previous_year2_CANTIDADCOMPRA
    if not previous_month_data.empty:
        previous_month_CANTIDADCOMPRA = previous_month_data.iloc[0]['CANTIDADCOMPRA']
        df.at[index, 'previous_month_CANTIDADCOMPRA'] = previous_month_CANTIDADCOMPRA
    if not previous_month_data2.empty:
        previous_month2_CANTIDADCOMPRA = previous_month_data2.iloc[0]['CANTIDADCOMPRA']
        df.at[index, 'previous_month2_CANTIDADCOMPRA'] = previous_month2_CANTIDADCOMPRA
    if not previous_month_data3.empty:
        previous_month3_CANTIDADCOMPRA = previous_month_data3.iloc[0]['CANTIDADCOMPRA']
        df.at[index, 'previous_month3_CANTIDADCOMPRA'] = previous_month3_CANTIDADCOMPRA
    if not next_year_data.empty:
        next_year_CANTIDADCOMPRA = next_year_data.iloc[0]['CANTIDADCOMPRA']
        df.at[index, 'next_year_CANTIDADCOMPRA'] = next_year_CANTIDADCOMPRA

df.head(2)


,year,month,ORIGEN,CODIGO,CANTIDADCOMPRA,previous_year_CANTIDADCOMPRA,previous_year2_CANTIDADCOMPRA,previous_month_CANTIDADCOMPRA,previous_month2_CANTIDADCOMPRA,previous_month3_CANTIDADCOMPRA,next_year_CANTIDADCOMPRA,date,covid
0,2015,1,0-0-2,B41691,100,0,0,0,0,0,100,2015-01-01,0.00
1,2015,1,0-0-2,E64663,250,0,0,0,0,0,300,2015-01-01,0.00


Creem 3 columnes noves amb els valors que formen la columna ORIGEN, per diferncciar entre regió hospital i departament.

Després eliminem la columna ORIGEN ja que amb les 3 noves seria redundància. També eliminem ORIGEN1 que correspon a la regió perquè hem vist qu no hi ha correlació amb lo q volem predir.

In [373]:
df[['ORIGEN1', 'ORIGEN2', 'ORIGEN3']] = df['ORIGEN'].str.split('-', n=2, expand=True)
df.drop(columns=['ORIGEN', 'ORIGEN1', 'date'], inplace=True)
df.head(2)


,year,month,CODIGO,CANTIDADCOMPRA,previous_year_CANTIDADCOMPRA,previous_year2_CANTIDADCOMPRA,previous_month_CANTIDADCOMPRA,previous_month2_CANTIDADCOMPRA,previous_month3_CANTIDADCOMPRA,next_year_CANTIDADCOMPRA,covid,ORIGEN2,ORIGEN3
0,2015,1,B41691,100,0,0,0,0,0,100,0.00,0,2
1,2015,1,E64663,250,0,0,0,0,0,300,0.00,0,2


Eliminem les dades del 2015 i 2016 perquè no podem mirar dos anys enrere.

In [374]:
df = df[(df['year']!=2015) & (df['year']!=2016)]
df.head(2)

,year,month,CODIGO,CANTIDADCOMPRA,previous_year_CANTIDADCOMPRA,previous_year2_CANTIDADCOMPRA,previous_month_CANTIDADCOMPRA,previous_month2_CANTIDADCOMPRA,previous_month3_CANTIDADCOMPRA,next_year_CANTIDADCOMPRA,covid,ORIGEN2,ORIGEN3
1582,2017,1,B41691,60,100,100,50,50,60,130,0.00,0,2
1583,2017,1,E64488,100,0,0,50,100,50,0,0.00,0,2


Canviem els tipus d'algunes columnes xq no dongui error.

In [375]:
cols = ['CODIGO', 'ORIGEN2', 'ORIGEN3', 'covid']
for c in cols:
    df[c] = df[c].astype('category')

cols = ['CANTIDADCOMPRA', 'previous_year_CANTIDADCOMPRA', 'previous_year2_CANTIDADCOMPRA', 'previous_month_CANTIDADCOMPRA', 'previous_month2_CANTIDADCOMPRA', 'next_year_CANTIDADCOMPRA']
for c in cols:
    df[c] = df[c].astype('float32')

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9134 entries, 1582 to 10715
Data columns (total 13 columns):
 #   Column                          Non-Null Count  Dtype   
---  ------                          --------------  -----   
 0   year                            9134 non-null   int32   
 1   month                           9134 non-null   int32   
 2   CODIGO                          9134 non-null   category
 3   CANTIDADCOMPRA                  9134 non-null   float32 
 4   previous_year_CANTIDADCOMPRA    9134 non-null   float32 
 5   previous_year2_CANTIDADCOMPRA   9134 non-null   float32 
 6   previous_month_CANTIDADCOMPRA   9134 non-null   float32 
 7   previous_month2_CANTIDADCOMPRA  9134 non-null   float32 
 8   previous_month3_CANTIDADCOMPRA  9134 non-null   int64   
 9   next_year_CANTIDADCOMPRA        9134 non-null   float32 
 10  covid                           9134 non-null   category
 11  ORIGEN2                         9134 non-null   category
 12  ORIGEN3              

### Split train i test

Creem dos datasets, el d'entrenament amb totes les dades excepte les de l'any 2022 i 2023. L'any 2023 no el volem fer servir per entrenar perquè CANTIDADCOMPRA és la que hem de predir i realment no podem tenir informació de les comandes abans de fer-les. L'altre dataset que creem és el de test que conté només les dades del 2022, per aicí poder predir les del 2023.

In [376]:
df_train = df[(df['year']!=2022)&(df['year']!=2023)].drop(columns='year')
df_test = df[df['year']==2022].drop(columns='year')

Dividim el dataset d'entrenament entre les variables d'entrenament i el target(CANTIDADCOMPRA de l'any següent)

In [377]:
df_train_X = df_train.drop(columns=['next_year_CANTIDADCOMPRA'])
df_train_y = df_train.loc[:,'next_year_CANTIDADCOMPRA']

Fem el mateix pel test

In [378]:
df_test_X = df_test.drop(columns=['next_year_CANTIDADCOMPRA'])
df_test_y = df_test.loc[:,'next_year_CANTIDADCOMPRA']

### Training

Utilitzem la llibreria xgboost, creem el model i l'entrenem.

In [379]:
import xgboost as xgb
# Use "hist" for training the model.
reg = xgb.XGBRegressor(tree_method="hist", enable_categorical = True)
# Fit the model using predictor X and response y.
reg.fit(df_train_X, df_train_y)
# Save model into JSON format.
reg.save_model("regressor.json")

### Predict

Fem la predicció

In [382]:
df_test['pred_next_year_CANTIDADCOMPRA'] = reg.predict(df_test_X)
df_test.loc[df_test["pred_next_year_CANTIDADCOMPRA"] < 0, "pred_next_year_CANTIDADCOMPRA"] = 0


Calculem dos metriques per veure com ho ha fet el model. Mirem rmse(root mean squared error)

In [386]:
np.sqrt(mean_squared_error(df_test["pred_next_year_CANTIDADCOMPRA"], df_test["next_year_CANTIDADCOMPRA"]))

0.05956279739713011

Comparem resultats de la predicció amb els resultats reals, per cada producte.

In [384]:
compare = df_test[["next_year_CANTIDADCOMPRA", "pred_next_year_CANTIDADCOMPRA", "CODIGO"]]

In [385]:
pd.options.display.float_format = '{:,.2f}'.format
compare.groupby("CODIGO").sum()

/var/folders/5y/ljztfs697fqf9rvmq2qtpg3m0000gn/T/ipykernel_30952/625031120.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  compare.groupby("CODIGO").sum()


,next_year_CANTIDADCOMPRA,pred_next_year_CANTIDADCOMPRA
CODIGO,,
B40558,141.00,318.05
B41691,"6,020.00","16,180.42"
C26183,0.00,304.95
C56207,0.00,473.51
E64488,"3,930.00","10,956.24"
E64543,"2,480.00","2,427.95"
E64544,"4,500.00","19,136.45"
E64663,"9,995.00","30,280.97"
E64750,"5,100.00","28,355.78"


Aquí podem veure com estem predint una quantitat total que quasi triplica la real:

In [388]:
print(df_test["next_year_CANTIDADCOMPRA"].sum())
print(df_test["pred_next_year_CANTIDADCOMPRA"].sum())

278135.0
694769.4


Per últim mirem la influència que té cada columna en la variable que volem predir.

In [389]:
p = {}
for i, j in zip(list(df_train_X.columns), list(reg.feature_importances_)):
    p[i] = j
p

{'month': 0.031130802,
 'CODIGO': 0.05450416,
 'CANTIDADCOMPRA': 0.5342778,
 'previous_year_CANTIDADCOMPRA': 0.12354128,
 'previous_year2_CANTIDADCOMPRA': 0.052911814,
 'previous_month_CANTIDADCOMPRA': 0.048890103,
 'previous_month2_CANTIDADCOMPRA': 0.040057935,
 'previous_month3_CANTIDADCOMPRA': 0.04794371,
 'covid': 0.01576868,
 'ORIGEN2': 0.04878698,
 'ORIGEN3': 0.0021866958}